# 1. 从Json种提取数据

In [1]:
import json

# 加载 JSON 文件
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# 提取对话内容
def extract_utterances(data):
    dialogues = []
    for dialogue in data:
        utterances = [turn['utterance'] for turn in dialogue['utterances']]
        dialogues.append(utterances)
    return dialogues

# 示例：加载和提取数据
file_path = '/opt/datasets/jpn_daily_dialog/topic1_raw.json'
data = load_json(file_path)
utterances = extract_utterances(data)

# 查看提取出的对话
for i in range(2):  # 查看前两条对话
    print(f"Dialogue {i+1}:")
    for utterance in utterances[i]:
        print(utterance)

Dialogue 1:
おはようございます。今日はとても良いお天気ですね。
はい。雲一つなくとても良い天気です。
この様子だと昼から暑くなりそうですね。
そうですね。お昼ご飯にはさっぱりした物が食べたくなります。
冷やし中華などでしょうか？
いいですね。よかったら一緒に食べに行きませんか？
ぜひ行きましょう。
Dialogue 2:
お昼休憩ですね。今日は何を食べますか？
今日はステーキが食べたい気分です。加藤さんは何が食べたいですか？
私はハンバーグが食べたいです。
では、ステーキとハンバーグの両方のメニューがあるお店を探しましょうか？
そうですね。この先にレストランのガストがあるので、ガストへ行きましょうか？
では、ガストへ行きましょう。私はドリンクバーの割引クーポンを持っているので、ドリンクバーもつけましょう。


# 2. 对日语进行分词

In [2]:
!pip3 install fugashi
!pip3 install unidic_lite

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
from transformers import AutoTokenizer

# 加载 BERT 分词器
# tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased") # For English
tokenizer = AutoTokenizer.from_pretrained("tohoku-nlp/bert-base-japanese") # For Japanese

# 对文本进行分词
text = "今日は天気がいいね！" # 如果使用【今日は天気がいいね！.】输出结果则是：['今日', 'は', '天気', 'が', 'いい', 'ね', '!', '##.']
tokens = tokenizer.tokenize(text)
print(tokens)

# 将分词结果转换为索引
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
print(indexed_tokens)  # 使用Bert可以直接将分词结果转换为索引，这样就不需要自己构建词表了

/home/vipuser/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['今日', 'は', '天気', 'が', 'いい', 'ね', '!']
[3246, 9, 11385, 14, 2575, 1852, 679]


# 3. 关于Bert分词的解释

如果使用下面的代码:


```python

from transformers import AutoTokenizer

# 加载 BERT 分词器
# tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased") # For English
tokenizer = AutoTokenizer.from_pretrained("tohoku-nlp/bert-base-japanese") # For Japanese

# 对文本进行分词
text = "今日は天気がいいね！."  
tokens = tokenizer.tokenize(text)
print(tokens) # ['今日', 'は', '天気', 'が', 'いい', 'ね', '!', '##.']

# 将分词结果转换为索引
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
print(indexed_tokens)
```
会输出:

```shell
['今日', 'は', '天気', 'が', 'いい', 'ね', '!', '##.']
```


**分析分词结果**

输出如下：

```shell
['今日', 'は', '天気', 'が', 'いい', 'ね', '!', '##.']
[3246, 9, 11385, 14, 2575, 1852, 679, 28651]
```

### **分词结果（tokens）**
- `'今日'`、`'は'`、`'天気'` 等这些都是分词器正常处理日语句子分出来的词元（tokens），这符合预期。
- `!`：标点符号 **`!`** 被正确识别为一个独立的词元，这是符合语言模型的设计，标点符号通常会被独立处理。
- `##.`：**`##.`** 是一个特殊的词元表示，它说明 `.`（句号）被认为是前面的标记的一部分，这是 BERT 使用**子词级分词**时常见的符号。前缀 `##` 表示这个词元是与前一个词连接的。

### **分词结果索引（indexed_tokens）**
- `[3246, 9, 11385, 14, 2575, 1852, 679, 28651]` 是词汇表中对应分词结果的索引值，这些值对应于 BERT 词汇表中的每个词元。

### 解释 **`##.`**
BERT 分词器使用**子词分割**（subword tokenization），即将不常见的词分割为更小的子词。前缀 `##` 表示这个子词不是单独的词，而是需要与前面的词组合起来。
- 在这个例子中，`##.` 表示句号 `.` 被作为**与前一个词结合的子词**。这是 BERT 分词器在处理词汇表中不存在的完整单词或符号时的常见现象。

#### 为什么会出现 **`##.`**
BERT 模型中的分词器采用的是 **WordPiece** 分词策略，这种策略将罕见的词（或符号）拆分为更小的子词（subword）。如果某个词汇（或标点）没有单独出现在词汇表中，BERT 会将它分割成子词，并使用 `##` 来表示这个子词是依附于前一个词的。
- 在这里，句号 `.` 可能没有被作为单独的词元出现，所以它被分为 `##.`，表示它是前一个词的一部分。

### 解决方法
如果你想避免出现像 `##.` 这样的子词连接符号，或希望句号 `.` 能独立分词，以下是两种可能的方案：

#### **允许更长的输入序列**
由于 `padding='max_length'` 和 `truncation=True` 可能会影响结果，你可以允许模型接收更长的输入以保留所有标点符号，避免它们被截断或与前面词元合并。
```python
tokens = tokenizer.tokenize(text, truncation=False)
```

#### **处理子词连接**
你可以在分词后处理带有 `##` 的子词。比如可以将这些子词合并为一个整体词元或去掉 `##` 前缀，确保句号被视为独立的词元。以下是处理的示例代码：

```python
processed_tokens = []
for token in tokens:
    if token.startswith("##"):
        processed_tokens[-1] += token[2:]  # 合并到前一个词元
    else:
        processed_tokens.append(token)

print(processed_tokens)
```

这种方式可以确保句号和其他标点符号被独立处理或与前面的词连接合适。

# 4. 关于Bert中特殊标记的解释

### 特殊标记的解释

1. **`[CLS]` 标记**：
   - BERT 模型的输入序列通常在开头会有一个特殊标记 `[CLS]`，它的索引值通常是 `2`。
   - **`[CLS]`** 用于表示整个句子的开始，并且在一些分类任务中，它的输出向量也会用于句子级别的预测。

2. **`[SEP]` 标记**：
   - **`[SEP]`** 标记用于句子之间的分隔，通常出现在句子的末尾。它的作用是在多句子任务（如自然语言推理）中，标记不同的句子边界。
   - 在单句任务中，它也常常出现在句子的末尾，用来标记句子的结束。

### 示例：BERT 分词的结果

假设输入是 `"今日は天気がいいね！"`，在使用 `tokenizer.encode()` 时，BERT 自动会加上这些特殊标记。输出的第一个 `2` 代表 `[CLS]` 标记，最后的 `3` 代表 `[SEP]` 标记。

```python
# 示例文本
text = "今日は天気がいいね！"

# 加载 BERT 日语分词器
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")

# 编码文本
encoded = tokenizer.encode(text, add_special_tokens=True)

# 输出编码结果
print(encoded)

# 解码回文本查看添加的特殊标记
decoded = tokenizer.convert_ids_to_tokens(encoded)
print(decoded)
```

### 示例输出：

```shell
[2, 3246, 9, 11385, 14, 2575, 1852, 8, 3]
['[CLS]', '今日', 'は', '天気', 'が', 'いい', 'ね', '！', '[SEP]']
```

- **`2`**：对应 `[CLS]`，代表句子的开始。
- **`3`**：对应 `[SEP]`，代表句子的结束。

### 特殊标记的用途

- **`[CLS]` 标记**：作为模型输入的起始标记，通常用于句子的全局表示。在某些任务（如文本分类）中，BERT 使用 `[CLS]` 的输出向量作为整个句子的表示。
- **`[SEP]` 标记**：作为句子的结束标记，用于区分多个句子或者标记单个句子的结束。

### 为什么所有句子都有 `2`？
这是因为 BERT 分词器默认会为每个句子添加 **`[CLS]`** 标记，这就是你看到每个句子开头都有 `2` 的原因。它不是分词结果的重复，而是 BERT 设计的特殊结构。

### 如何避免这些特殊标记？

如果你不想要这些特殊标记，可以将 `add_special_tokens=False` 传递给 `tokenizer.encode()`，这样就不会添加 `[CLS]` 和 `[SEP]`。

```python
encoded_no_special = tokenizer.encode(text, add_special_tokens=False)
print(encoded_no_special)
```

### 示例输出（不带特殊标记）：

```shell
[3246, 9, 11385, 14, 2575, 1852, 8]
```

这时就只剩下句子本身的分词结果，没有添加 `[CLS]` 和 `[SEP]` 标记。